In [ ]:
#神经网络模型
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import load_model

CAND = 12
BATCH_SIZE = 64
NUM_CLASSES = 4
NUM_EPOCHS = 10

##搭建神经网络
model = Sequential()
# 输入: CAND 个通道 4x4 棋盘 -> (4, 4, CAND) 张量。
model.add(Conv2D(128, (4, 1), padding='same', activation='relu', kernel_initializer='he_uniform', input_shape=(4, 4, CAND)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (1, 4), padding='same',  kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2), padding='same',  kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding='same',  kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (4, 4), padding='same',  kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())#展成一维

model.add(Dense(128,  kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128,  kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(4,  kernel_initializer='he_uniform',activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

In [ ]:
from keras.models import load_model
import keras
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent,MyAgent
import numpy as np

for k in range(3):

    #提取训练数据X,Y
    Num = 50
    X = np.zeros((0,4,4))
    Y = np.zeros((0))
    for i in range(Num):
        game = Game(4, score_to_win=4096, random=False)
        agent = MyAgent(game)
        X1,Y1 = agent.play(verbose=False)  #每次循环的结果
        X = np.concatenate((X,X1),axis=0)
        Y = np.concatenate((Y,Y1),axis=0)
        if (i%10)==0:
            print(i)

    print(X.shape,Y.shape)
    
    #生成字典{0:0, 2:1, 4:2,..., 2048:11, 4096:12}
    OUT_SHAPE = (4,4)
    CAND = 12
    map_table = {2**i: i for i in range(1, CAND)}
    map_table[0] = 0

    ##将4x4的棋盘表示为one-hot的形式，有三个维度，行、列和每个格子的数值
    def grid_ohe(arr):
        ret = np.zeros(shape=OUT_SHAPE + (CAND,))
        for r in range(OUT_SHAPE[0]):
            for c in range(OUT_SHAPE[1]):
                ret[r,c,map_table[arr[r,c]]] = 1
        return ret


    ##处理数据X，使其变为4x4xCAND
    X_train = np.ones((X.shape[0], 4, 4, CAND))

    for i in range(X.shape[0]):
        X_train[i] = grid_ohe(X[i])

    ##处理数据Y  convert class vectors to binary class matrics
    Y_train = keras.utils.to_categorical(Y, 4)

    print(X_train.shape,Y_train.shape)

    model = load_model('model.h5')
    for i in range(5):
        model.fit(X_train, Y_train, batch_size = 64 , epochs=5)
        model.save('model.h5')